In [24]:
from data_preprocess import DataPreprocess
import pandas as pd
import numpy as np

In [17]:
df = DataPreprocess('ETHBTC-5m-data.csv').get_ohlcv()

In [22]:
window_size = 12
horizon = 1

In [23]:
# Add window columns
ETHBTC_windowed = df.copy()
for i in range(window_size):
    ETHBTC_windowed[f'Close+{i+1}'] = ETHBTC_windowed['Close'].shift(periods=i+1)

ETHBTC_windowed.head()

,Open,High,Low,Close,Volume,Close+1,Close+2,Close+3,Close+4,Close+5,Close+6,Close+7,Close+8,Close+9,Close+10,Close+11,Close+12
Timestamp,,,,,,,,,,,,,,,,,
2017-07-14 04:00:00,0.080000,0.080000,0.080000,0.080000,0.726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-14 04:05:00,0.080000,0.080001,0.080000,0.080001,3.347,0.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-14 04:10:00,0.080001,0.086400,0.080001,0.086400,4.679,0.080001,0.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-14 04:15:00,0.085289,0.085620,0.085128,0.085128,53.431,0.086400,0.080001,0.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-14 04:20:00,0.085274,0.086000,0.085274,0.086000,5.576,0.085128,0.086400,0.080001,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Create feature data and target data
X = ETHBTC_windowed.dropna().drop('Close', axis=1).astype(np.float32)
y = ETHBTC_windowed.dropna()['Close'].astype(np.float32)
X.head(), y.head()

(                         Open      High       Low  Volume   Close+1   Close+2  \
 Timestamp                                                                       
 2017-07-14 05:00:00  0.085874  0.086205  0.085874   1.304  0.086347  0.086380   
 2017-07-14 05:05:00  0.085816  0.085816  0.084824   9.770  0.085978  0.086347   
 2017-07-14 05:10:00  0.084803  0.084803  0.084608   5.446  0.084824  0.085978   
 2017-07-14 05:15:00  0.084581  0.086196  0.084581   8.833  0.084680  0.084824   
 2017-07-14 05:20:00  0.085590  0.085810  0.085496   4.461  0.086196  0.084680   
 
                       Close+3   Close+4   Close+5   Close+6   Close+7  \
 Timestamp                                                               
 2017-07-14 05:00:00  0.086350  0.086314  0.086180  0.086000  0.085811   
 2017-07-14 05:05:00  0.086380  0.086350  0.086314  0.086180  0.086000   
 2017-07-14 05:10:00  0.086347  0.086380  0.086350  0.086314  0.086180   
 2017-07-14 05:15:00  0.085978  0.086347  0.086380  0.

In [26]:
# Make train_test_split
train_size = int(len(X)*0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

(362858, 362858, 90715, 90715)

In [29]:
import os

# Create a function to implement a ModelCheckpoint callback with a specific filename
def create_model_checkpoint(model_name, save_path='model_experiments'):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name),
                                            verbose=0, # only output text when model is saved
                                            monitor='val_loss',
                                            save_best_only=True)

In [30]:
# Build an MLP baseline model using tensorflow

import tensorflow as tf
from tensorflow.keras import layers

model_1 = tf.keras.Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(horizon)
], name='baseline_model')

model_1.compile(loss='mae', optimizer='adam')
model_1.fit(X_train, y_train,
            epochs=100,
            validation_data=(X_test, y_test),
            verbose=0,
            callbacks=[create_model_checkpoint(model_name=model_1.name)])

INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets
INFO:tensorflow:Assets written to: model_experiments\baseline_model\assets


In [31]:
def evaluate_preds(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    
    # Calculate metrics
    mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
    mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
    rmse = tf.sqrt(mse)
    mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
    
    return {'mae': mae.numpy(),
            'mse': mse.numpy(),
            'rmse': rmse.numpy(),
            'mape': mape.numpy()}

In [32]:
model_1 = tf.keras.models.load_model('model_experiments/baseline_model/')

In [33]:
model_1.evaluate(X_test, y_test)

2835/2835 [==============================] - 7s 2ms/step - loss: 1.2316e-04


0.0001231614878633991

In [34]:
model_1_preds = tf.squeeze(model_1.predict(X_test))
model_1_preds[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.02694072, 0.0269377 , 0.02693159, 0.02690902, 0.02691715],
      dtype=float32)>

In [36]:
evaluate_preds(y_test, model_1_preds)

{'mae': 0.0001231614,
 'mse': 3.7594226e-08,
 'rmse': 0.00019389231,
 'mape': 0.23426421}